# Dogs vs Cats Kernels Redux Edition

Here I am using VGG16 model which has won the imagenet competition a few years back. The utils files are just utility fucntions written for our benefit and has been taken as it is from fast.ai course.

In [1]:
%matplotlib inline

In [10]:
#path = "data/cats-dogs-redux/sample/"           #path to sample data
#path = "data/cats-dogs-redux/"                  #path to entire data
test_path = "data/cats-dogs-redux/test/"         #path to test data

In [3]:
# importing required libraries
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
import pandas as pd
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:
from importlib import reload      #py3
import utils; reload(utils)
from utils import plots

Using cuDNN version 6021 on context None
Mapped name None to device cuda0: Tesla K80 (EE11:00:00.0)
Using Theano backend.


In [5]:
#batch size can start from 2, then 4, then 8..so on untill 64 (afterwards its not advisable)
batch_size=64

In [6]:
# Import Vgg class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [7]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 1125 images belonging to 2 classes.
Found 125 images belonging to 2 classes.
Epoch 1/1
1125/1125 [==============================] - 29s - loss: 0.4776 - acc: 0.8524 - val_loss: 0.0077 - val_acc: 1.0000


## Training model on entire data

In [9]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=3)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Epoch 1/3
22500/22500 [==============================] - 545s - loss: 0.1262 - acc: 0.9684 - val_loss: 0.0675 - val_acc: 0.9832
Epoch 2/3
22500/22500 [==============================] - 543s - loss: 0.0969 - acc: 0.9772 - val_loss: 0.0608 - val_acc: 0.9848
Epoch 3/3
22500/22500 [==============================] - 546s - loss: 0.0918 - acc: 0.9787 - val_loss: 0.0708 - val_acc: 0.9844


In [11]:
def get_test_batches(size = 64):
    batches = vgg.get_batches(test_path, batch_size=size, shuffle=False)
    return batches

In [12]:
def get_test_filename(batches, i):
    return batches.filenames[i][8:-4]

In [13]:
def get_test_rec(batches):
    i = 0
    records = []
    for batch in batches:
        imgs, labels = batch
        probs, class_ids, class_names = vgg.predict(imgs, True)
        for prob, class_id in zip(probs, class_ids):
            if i >= len(batches.filenames):
                return records
            prob_dog = prob
            if class_id == 0:
                prob_dog = 1 - prob
            filename = get_test_filename(batches, i)
            records.append({'id' : filename ,  'label' : format(prob_dog, '.8f') })    
            i = i + 1
            if i % 1000 == 0:
                print(i)
        if i >= len(batches.filenames):
            break
    return records

In [14]:
def write_submission_csv(records):
    df = pd.DataFrame.from_records(records)
    df['id'] = pd.to_numeric(df['id'])
    df = df.sort_values('id')
    df.to_csv('new_submission.csv', index=False)

In [15]:
testbatches = get_test_batches()
records = get_test_rec(testbatches)
write_submission_csv(records=records)

Found 12500 images belonging to 1 classes.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [16]:
path = "data/"
from IPython.display import FileLink
FileLink(path + 'new_submission.csv')

/home/ubuntu/cats-vs-dogs-redux-edition/data/new_submission.csv